In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/in_process/Popular_Games_expanded.csv', index_col = 'game_id')
dfgood = pd.read_csv('../data/in_process/Popular_Games.csv', index_col = 'game_id')

In [3]:
df.describe()

,min_players,max_players,avg_time,year,avg_rating,geek_rating,num_votes,age,owned,weight
count,4768.000000,4768.000000,4768.000000,4768.000000,4768.000000,4768.000000,4768.000000,4768.000000,4768.000000,4768.000000
mean,1.998951,5.466862,82.654308,1996.456586,6.950445,6.245841,3084.935612,10.657508,5277.257550,2.273101
std,0.699575,7.197982,246.712028,175.123777,0.771389,0.539750,6084.947744,2.741033,8597.807335,0.776210
min,0.000000,0.000000,1.000000,-3500.000000,2.266230,3.526090,500.000000,0.000000,263.000000,1.000000
25%,2.000000,4.000000,30.000000,2004.000000,6.485913,5.874825,750.000000,8.000000,1616.750000,1.673400
50%,2.000000,5.000000,60.000000,2011.000000,6.964510,6.161935,1238.500000,10.000000,2778.500000,2.217450
75%,2.000000,6.000000,90.000000,2015.000000,7.485098,6.550277,2663.000000,13.000000,5382.000000,2.786300
max,8.000000,100.000000,12000.000000,2020.000000,9.186900,8.576860,93524.000000,18.000000,139881.000000,4.723300


##Observations
Number of votes highly correlated with number of users who own the game. Makes sense.
Number of votes also associated with "geek rating." Makes sense because Geek Rating is a Bayesian average.
Number owned is correlated with both ratings. Makes sense that people would vote for stuff they own.
Average playing time and "weight" are correlated. Makes sense. Longer games are weightier.
Recommended age and wieght are correlated because games for kids are less weighty.


In [4]:
#Dropping irrelevant games
df = df.drop([18291, 21804, 23953], axis = 0)

In [5]:
#looking up these two on the web site to get correct player counts
df.loc[4149, 'max_players'] = 1
df.loc[25738, 'max_players'] = 4
df.loc[177497, 'max_players'] = 4
df.loc[177497, 'min_players'] = 1

In [6]:
#just drop them if we want minimum age to be a relevant factor
dfage=df[df['age'] == 0]
dfage = dfage[['age', 'weight']]
df = df.drop(dfage.index, axis = 0)

In [7]:
#limit examination to only more modern games
#will define that as games released since 1960
dfy = pd.DataFrame(df['year'])
dfy2 = dfy[dfy['year'] < 1960]
df = df.drop(dfy2.index, axis = 0)
df.describe()

,min_players,max_players,avg_time,year,avg_rating,geek_rating,num_votes,age,owned,weight
count,4630.000000,4630.000000,4630.000000,4630.000000,4630.000000,4630.000000,4630.000000,4630.000000,4630.000000,4630.000000
mean,1.999136,5.457451,83.016465,2008.392009,6.966894,6.258314,3087.955508,10.858963,5288.644276,2.278022
std,0.699658,7.024371,250.067805,10.061053,0.739821,0.529486,6112.668181,2.420251,8625.648781,0.774208
min,1.000000,1.000000,1.000000,1960.000000,3.331770,4.272580,500.000000,2.000000,263.000000,1.000000
25%,2.000000,4.000000,30.000000,2005.000000,6.496457,5.880187,751.000000,9.000000,1634.000000,1.680100
50%,2.000000,5.000000,60.000000,2011.000000,6.971080,6.169290,1249.000000,11.000000,2794.000000,2.222200
75%,2.000000,6.000000,90.000000,2015.000000,7.487145,6.560507,2679.750000,13.000000,5402.750000,2.787725
max,8.000000,100.000000,12000.000000,2020.000000,9.186900,8.576860,93524.000000,18.000000,139881.000000,4.723300


### Observations

##### Number of votes highly correlated with number of users who own the game. Makes sense.
##### Number of votes also associated with "geek rating." Makes sense because Geek Rating is a Bayesian average.
##### Number owned is correlated with both ratings. Makes sense that people would vote for stuff they own.
##### Average playing time and "weight" are correlated. Makes sense. Longer games are weightier.
##### Recommended age and wieght are correlated because games for kids are less weighty.
#####  Average rating and min age may be correlated. Do people like games that can be played with their kids?

In [8]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing

In [9]:
df = df.drop(['num_votes', 'owned'], axis = 1)


In [10]:
#dropping expansions and just looking at original games
df = df[df['expands'] == 'Nothing']

In [11]:
df = df.drop(['names', 'designer', 'publisher', 'expands'], axis = 1)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3836 entries, 1 to 287954
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   min_players  3836 non-null   int64  
 1   max_players  3836 non-null   int64  
 2   avg_time     3836 non-null   float64
 3   year         3836 non-null   int64  
 4   avg_rating   3836 non-null   float64
 5   geek_rating  3836 non-null   float64
 6   age          3836 non-null   int64  
 7   mechanic     3836 non-null   object 
 8   category     3836 non-null   object 
 9   weight       3836 non-null   float64
 10  Category_1   3836 non-null   object 
 11  Category_2   3214 non-null   object 
 12  Category_3   2111 non-null   object 
 13  Category_4   1098 non-null   object 
 14  Mechanic_1   3836 non-null   object 
 15  Mechanic_2   3355 non-null   object 
 16  Mechanic_3   2595 non-null   object 
 17  Mechanic_4   1789 non-null   object 
 18  Mechanic_5   1135 non-null   object 
dtypes: f

In [13]:
dfexpandedC = df.iloc[:, 10:14]

In [14]:
dfexpandedM = df.iloc[:, 14:]

In [15]:
dfexpandedM.head()

,Mechanic_1,Mechanic_2,Mechanic_3,Mechanic_4,Mechanic_5
game_id,,,,,
1,Area Majority / Influence,Auction/Bidding,Dice Rolling,Hand Management,Simultaneous Action Selection
2,Trick-taking,NaN,NaN,NaN,NaN
3,Area Majority / Influence,Hand Management,Set Collection,Tile Placement,NaN
5,Hand Management,Investment,Market,Ownership,Stock Holding
7,Enclosure,Pattern Building,Pattern Recognition,Tile Placement,NaN


In [16]:
#This just gives us a list of all categories
Cats1 = df['category'].str.split(pat=', ', expand=False).tolist()
Cats = list()
for i in range(len(Cats1)):
    for j in range(len(Cats1[i])):
        Cats.append(Cats1[i][j])

In [17]:
#this gives us a list of all mechanics
Mech1 = df['mechanic'].str.split(pat=', ', expand=False).tolist()
Mech = list()
for i in range(len(Mech1)):
    for j in range(len(Mech1[i])):
        Mech.append(Mech1[i][j])

In [18]:
Cat_counts = dfexpandedC.apply(pd.Series.value_counts).sum(axis=1)

In [19]:
Mech_counts = dfexpandedM.apply(pd.Series.value_counts).sum(axis = 1)

In [20]:
df_C = pd.DataFrame()
for catg in range(len(Cats)):
    df_C[Cats[catg]] = df['category'].str.contains(Cats[catg]).astype(int)

In [21]:
for i in df_C.columns:
    df_C[i] = df_C[i]*Cat_counts[i]

In [22]:
df_M = pd.DataFrame()
for mechs in range(len(Mech)):
    df_M[Mech[mechs]] = df['mechanic'].str.contains(Mech[mechs]).astype(int)


In [23]:
#When this was first run, df_M has a column "order counters"
#This value was not in the counts. It must not have been one of the first 5
#Mechanics listed in any one row. Rather than go back and look for it, I'm going
#to just drop it.
df_M = df_M.drop(['Order Counters'], axis = 1)

In [24]:
for j in df_M.columns:
    df_M[j] = df_M[j]*Mech_counts[j]

In [25]:
df_M.head()

,Area Majority / Influence,Auction/Bidding,Dice Rolling,Hand Management,Simultaneous Action Selection,Trick-taking,Set Collection,Tile Placement,Investment,Market,...,Automatic Resource Growth,Prisoner's Dilemma,Narrative Choice / Paragraph,Contracts,Moving Multiple Units,King of the Hill,Force Commitment,Legacy Game,Bingo,Pattern Movement
game_id,,,,,,,,,,,,,,,,,,,,,
1,564.0,309.0,1055.0,1153.0,254.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,564.0,0.0,0.0,1153.0,0.0,0.0,694.0,399.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1153.0,0.0,0.0,0.0,399.0,10.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,399.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Got all categories and Mechanics to be Count Encoded.
Need to merge these columns back into the original dataframe. Drop the non needed stuff,
and then try the regressions again.

In [26]:
#with the above columns for each element of Category and Mechanic
#we don't need the split out columns
df = df.drop(['Category_1', 'Category_2', 'Category_3', 'Category_4'], axis = 1)
df = df.drop(['Mechanic_1', 'Mechanic_2', 'Mechanic_3', 'Mechanic_4', 'Mechanic_5'], axis = 1)

In [27]:
df = df.drop(['mechanic', 'category'], axis = 1)

In [28]:
df.describe()

,min_players,max_players,avg_time,year,avg_rating,geek_rating,age,weight
count,3836.000000,3836.000000,3836.000000,3836.000000,3836.000000,3836.000000,3836.000000,3836.000000
mean,2.010688,5.436131,82.510167,2007.887904,6.834371,6.222357,10.722888,2.217682
std,0.681773,7.278325,273.188526,10.636066,0.698512,0.539138,2.427055,0.782007
min,1.000000,1.000000,1.000000,1960.000000,3.331770,4.272580,2.000000,1.000000
25%,2.000000,4.000000,30.000000,2004.000000,6.421880,5.844862,8.000000,1.590000
50%,2.000000,5.000000,60.000000,2011.000000,6.848635,6.109750,10.000000,2.150900
75%,2.000000,6.000000,90.000000,2015.000000,7.300432,6.523722,12.000000,2.730650
max,8.000000,100.000000,12000.000000,2020.000000,9.186900,8.576860,18.000000,4.723300


In [29]:
df = df.join(df_M, how = 'left')

In [32]:
df = df.join(df_C, how = 'left', rsuffix = 'cat_')

In [34]:
#Using OLS without train/test splitting just to get an overall
#view of what a regression would look like.
X = df.drop(['avg_rating', 'geek_rating'], axis = 1)
y1 = df['avg_rating']
y2 = df['geek_rating']
X = sm.add_constant(X)

In [35]:
#Basic linear model on the average user rating
rModel1 = sm.OLS(y1, X)
rModel1_result = rModel1.fit()
rModel1_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             avg_rating   R-squared:                       0.599
Model:                            OLS   Adj. R-squared:                  0.568
Method:                 Least Squares   F-statistic:                     19.57
Date:                Fri, 30 Jul 2021   Prob (F-statistic):               0.00
Time:                        18:06:32   Log-Likelihood:                -2313.3
No. Observations:                3836   AIC:                             5173.
Df Residuals:                    3563   BIC:                             6880.
Df Model:                         272                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                  -41.0869      1.766    -23.260      0.000     -44.550     -37.624
min_players                             -0.0124      0.015     -0.818      0.413      -0.042       0.017
max_players                             -0.0003      0.001     -0.310      0.757      -0.003       0.002
avg_time                              2.633e-05   3.28e-05      0.803      0.422    -3.8e-05    9.06e-05
year                                     0.0233      0.001     26.450      0.000       0.022       0.025
age                                      0.0018      0.004      0.412      0.681      -0.007       0.011
weight                                   0.4167      0.018     22.769      0.000       0.381       0.453
Area Majority / Influence            -2.567e-05   4.49e-05     -0.571      0.568      -0.000    6.24e-05
Auction/Bidding                         -0.0003      0.000     -2.587      0.010      -0.000   -6.36e-05
Dice Rolling                          1.354e-06    2.3e-05      0.059      0.953   -4.38e-05    4.65e-05
Hand Management                       3.315e-05   1.68e-05      1.977      0.048    2.79e-07     6.6e-05
Simultaneous Action Selection            0.0003      0.000      2.547      0.011    7.22e-05       0.001
Trick-taking                             0.0024      0.001      1.898      0.058   -7.97e-05       0.005
Set Collection                        5.695e-05   3.08e-05      1.851      0.064   -3.39e-06       0.000
Tile Placement                        6.385e-05   6.84e-05      0.934      0.350   -7.02e-05       0.000
Investment                               0.0258      0.016      1.629      0.103      -0.005       0.057
Market                                   0.0059      0.005      1.099      0.272      -0.005       0.016
Ownership                                0.0087      0.008      1.073      0.283      -0.007       0.025
Stock Holding                        -7.153e-05      0.001     -0.060      0.952      -0.002       0.002
Enclosure                               -0.0013      0.001     -1.146      0.252      -0.003       0.001
Pattern Building                         0.0004      0.000      1.153      0.249      -0.000       0.001
Pattern Recognition                     -0.0015      0.001     -1.741      0.082      -0.003       0.000
Card Drafting                        -6.354e-05      0.000     -0.301      0.763      -0.000       0.000
Network and Route Building               0.0004      0.000      1.702      0.089   -5.45e-05       0.001
Point to Point Movement               9.441e-05      0.000      0.537      0.591      -0.000       0.000
Negotiation                              0.0660      0.037      1.796      0.073      -0.006       0.138
Trading                               

In [36]:
#Basic linear model on the geek rating
rModel2 = sm.OLS(y2, X)
rModel2_result = rModel2.fit()
rModel2_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            geek_rating   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.465
Method:                 Least Squares   F-statistic:                     13.26
Date:                Fri, 30 Jul 2021   Prob (F-statistic):               0.00
Time:                        18:07:45   Log-Likelihood:                -1731.6
No. Observations:                3836   AIC:                             4009.
Df Residuals:                    3563   BIC:                             5716.
Df Model:                         272                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                  -11.8828      1.518     -7.829      0.000     -14.859      -8.907
min_players                             -0.0032      0.013     -0.249      0.803      -0.029       0.022
max_players                             -0.0001      0.001     -0.119      0.905      -0.002       0.002
avg_time                             -2.768e-05   2.82e-05     -0.982      0.326   -8.29e-05    2.76e-05
year                                     0.0087      0.001     11.447      0.000       0.007       0.010
age                                     -0.0039      0.004     -1.028      0.304      -0.011       0.004
weight                                   0.2262      0.016     14.380      0.000       0.195       0.257
Area Majority / Influence             1.425e-05   3.86e-05      0.369      0.712   -6.14e-05    8.99e-05
Auction/Bidding                      -9.035e-05   8.72e-05     -1.036      0.300      -0.000    8.07e-05
Dice Rolling                          3.762e-05   1.98e-05      1.903      0.057   -1.14e-06    7.64e-05
Hand Management                       9.506e-05   1.44e-05      6.599      0.000    6.68e-05       0.000
Simultaneous Action Selection            0.0004      0.000      3.716      0.000       0.000       0.001
Trick-taking                             0.0016      0.001      1.456      0.145      -0.001       0.004
Set Collection                           0.0001   2.64e-05      3.858      0.000    5.02e-05       0.000
Tile Placement                        3.065e-05   5.87e-05      0.522      0.602   -8.45e-05       0.000
Investment                               0.0516      0.014      3.793      0.000       0.025       0.078
Market                                   0.0045      0.005      0.968      0.333      -0.005       0.014
Ownership                                0.0134      0.007      1.916      0.055      -0.000       0.027
Stock Holding                           -0.0018      0.001     -1.793      0.073      -0.004       0.000
Enclosure                               -0.0007      0.001     -0.684      0.494      -0.003       0.001
Pattern Building                         0.0006      0.000      1.951      0.051   -2.89e-06       0.001
Pattern Recognition                     -0.0011      0.001     -1.519      0.129      -0.003       0.000
Card Drafting                           -0.0001      0.000     -0.672      0.501      -0.000       0.000
Network and Route Building               0.0002      0.000      1.284      0.199      -0.000       0.001
Point to Point Movement                  0.0002      0.000      1.070      0.284      -0.000       0.000
Negotiation                              0.0878      0.032      2.780      0.005       0.026       0.150
Trading                               

In [37]:
df.to_csv('../data/in_process/Games_CountEncoded.csv')
